In [ ]:
import numpy as np

In [ ]:
from NeuronalNetwork import NeuralLayer, NeuralNetwork
from LossFunc import binary_crossentropy_derivative, binary_crossentropy
from ActivFunc import relu, relu_derivative, softmax, softmax_derivative
from Optimizer import SGD
import Datasets
from MetricUtils import display_loss_per_epoch, accuracy, confusionMatrix

In [ ]:
def build_neural_network(X_train, y_train):
    # Arquitectura de la red neuronal
    input_size = X_train.shape[1]
    output_size = y_train.shape[1]
    hidden_size1 = 100
    hidden_size2 = 100

    # Creación de capas
    input_layer = NeuralLayer(input_size, hidden_size1, relu, relu_derivative)
    hidden_layer1 = NeuralLayer(hidden_size1, hidden_size2, relu, relu_derivative)
    hidden_layer2 = NeuralLayer(hidden_size2, output_size, softmax, softmax_derivative, use_bias=False)

    # Creación de la red neuronal
    neural_network = NeuralNetwork([input_layer, hidden_layer1, hidden_layer2])
    return neural_network

In [ ]:
def train_neural_network(neural_network, X_train, y_train, X_test, y_test, optimizer, epochs=100, batch_size=20, momentum=0.9):
    
    loss_per_epoch = list()
    
    # Entrenamiento
    for epoch in range(epochs):
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices) # reorganiza de manera aleatoria el indice
        

        for i in range(0, X_train.shape[0], batch_size):
            batch_indices = indices[i:i+batch_size]
            X_batch = X_train[batch_indices]
            y_batch = y_train[batch_indices]

            # Forward pass
            predicted_output = neural_network.forward(X_batch)

            # Cálculo de la pérdida
            loss = np.mean(binary_crossentropy(y_batch, predicted_output))

            # Backpropagation
            error = binary_crossentropy_derivative(y_batch, predicted_output)
            neural_network.backward(error, optimizer, momentum)

        # Evaluación en el conjunto de prueba
        test_output = neural_network.forward(X_test)
        test_loss = np.mean(binary_crossentropy(y_test, test_output))
        loss_per_epoch.append(test_loss)

        print(f"Epoch {epoch + 1} -  Test Loss: {test_loss:.4f}")
    
    display_loss_per_epoch(loss_per_epoch)


In [ ]:
# Carga y preprocesamiento de datos
X_train, y_train, X_test, y_test = Datasets.load_and_preprocess_data_mnist("../data") # añadir direccion de la carpeta con datos MNIST

#X_train, y_train, X_test, y_test = Datasets.load_and_preprocess_data_iris()

# Construcción de la red neuronal
neural_network = build_neural_network(X_train, y_train)

# Entrenamiento de la red neuronal
optimizer = SGD(learning_rate=0.01)
train_neural_network(neural_network, X_train, y_train, X_test, y_test, optimizer, epochs=50, batch_size=150, momentum=0.9)

y_pred = neural_network.forward(X_test)

# Presición
print(f"Accuracy on Test Set: {accuracy(y_test, y_pred):.2f}%")

# Matriz de confusion
confusionMatrix(y_test, y_pred)